<a href="https://colab.research.google.com/github/agntgalahad/pothole-detection/blob/main/ResNet_Pothole_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, RandomRotation, RandomFlip
from keras.applications import MobileNet
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d atulyakumar98/pothole-detection-dataset

pothole-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
!mkdir dataset
with zipfile.ZipFile('/content/pothole-detection-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset/')

mkdir: cannot create directory ‘dataset’: File exists


In [4]:
from pathlib import Path
import imghdr
import os

img_link=list(Path("/content/dataset/normal").glob(r'**/*.jpg'))

count_num=0
for lnk in img_link:
    binary_img=open(lnk,'rb')
    find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
    if not find_img:
        count_num+=1
        os.remove(str(lnk))
print('Total %d pcs image delete from Dataset' % count_num)

Total 59 pcs image delete from Dataset


In [5]:
img_link=list(Path("/content/dataset/potholes").glob(r'**/*.jpg'))

count_num=0
for lnk in img_link:
    binary_img=open(lnk,'rb')
    find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
    if not find_img:
        count_num+=1
        os.remove(str(lnk))
print('Total %d pcs image delete from Dataset' % count_num)

Total 31 pcs image delete from Dataset


In [6]:

img_height, img_width = 227, 227
batch_size = 32
data_dir = '/content/dataset'
class_names = ['normal', 'potholes']

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'training',
    label_mode = 'binary',
    batch_size = batch_size,
    seed = 123,
    image_size=(img_height, img_width),
)
print(train_ds)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = 'validation',
    label_mode = 'binary',
    batch_size = batch_size,
    seed = 123,
    image_size=(img_height, img_width),
)
print(valid_ds)

Found 591 files belonging to 2 classes.
Using 473 files for training.
<BatchDataset element_spec=(TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>
Found 591 files belonging to 2 classes.
Using 118 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 227, 227, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


In [7]:
model = Sequential()

pre_model = keras.applications.ResNet50(include_top = False,
                                        input_shape = (227,227,3),
                                        pooling = 'avg',
                                        classes = 2,
                                        weights = 'imagenet')

for layer in pre_model.layers:
  layer.trainable = False

model.add(RandomFlip('horizontal_and_vertical'))
model.add(pre_model)
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(2, activation = 'sigmoid'))
model.build((None,227,227,3))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 227, 227, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


In [9]:
history = model.fit(train_ds, batch_size=batch_size, epochs = 5)

Epoch 1/5
15/15 [==============================] - 17s 386ms/step - loss: 0.4741 - accuracy: 0.8224
Epoch 2/5
15/15 [==============================] - 6s 231ms/step - loss: 0.1875 - accuracy: 0.9408
Epoch 3/5
15/15 [==============================] - 6s 229ms/step - loss: 0.0646 - accuracy: 0.9746
Epoch 4/5
15/15 [==============================] - 6s 227ms/step - loss: 0.0288 - accuracy: 0.9915
Epoch 5/5
15/15 [==============================] - 11s 371ms/step - loss: 0.0149 - accuracy: 0.9958


In [10]:
test = model.evaluate(valid_ds, batch_size = 32, verbose = 2)

4/4 - 3s - loss: 0.0868 - accuracy: 0.9576 - 3s/epoch - 637ms/step


In [11]:
!mkdir saved_models
model.save('/content/saved_models')